# Transform Stage: Account Endpoints

In [1]:
import json
from datetime import datetime
from pathlib import Path
from typing import Any, Self, TypeAlias

import pandas as pd
from pandas import DataFrame
from pydantic import BaseModel

# from rich.pretty import pprint

## Helper Functions
### Read API Data Function

In [2]:
def read_api_data(endpoint_name: str, data_dir: Path) -> dict[str, Any]:
    """Read an API response's JSON from file.

    Parameters
    ----------
    endpoint_name : str
        API endpoint name.
    data_dir : Path
        Path to the data directory for storing the output JSON file.

    Returns
    -------
    dict[Any, Any]
        JSON returned as a dict.
    """
    # location of the input JSON file.
    file_path = data_dir.joinpath(f"{endpoint_name}.json")
    with open(file_path, "r") as f:
        data = json.load(f)
    return data

### Split History of Transactions

In [3]:
# Transaction container
TC: TypeAlias = dict[str, Any]


def split_history(history_raw: dict[str, Any]) -> tuple[TC, TC, TC]:
    """Split transactions based on transaction type.

    Transaction types are as follows:
    - equity
    - options
    - overrides (transactions external to tradier.com)
    - other (remaining transactions that do not fit the categories above)

    Parameters
    ----------
    history_raw : dict[str, Any]
        Raw data from history endpoint

    Returns
    -------
    tuple[TC, TC, TC, TC]
        Transaction containers (TC) for each transaction type.
    """
    equity_raw: list[dict[str, Any]] = []
    options_raw: list[dict[str, Any]] = []
    overrides_raw: list[dict[str, Any]] = []
    other_raw: list[dict[str, Any]] = []
    for transaction in history_raw:
        if transaction["type"] == "trade":
            if transaction["trade"]["trade_type"] == "equity":
                equity_raw.append(transaction)
            elif transaction["trade"]["trade_type"] == "option":
                options_raw.append(transaction)
            else:
                other_raw.append(transaction)
        elif transaction["type"] == "option":
            overrides_raw.append(transaction)
        else:
            other_raw.append(transaction)
    tradier_trades_raw = {"equity": equity_raw, "options": options_raw}
    return tradier_trades_raw, overrides_raw, other_raw

## Pydantic Model
### Transactions: Equities and Options

- Raw transaction structure:
    - `amount` - float
    - `date` - datetime
    - `type` - str
    - `trade` - dict
        - `commission` - float
        - `description` - str
        - `price` - float
        - `quantity` - float
        - `symbol` - str
        - `trade_type` - str

<br>

- Flat transaction structure:
    - `date` - datetime
    - `trade_type` - str
    - `description` - str
    - `symbol` - str
    - `price` - float
    - `quantity` - float
    - `amount` - float
    - `commission` - float

In [4]:
class RawTrade(BaseModel):
    """Raw trade fields."""

    commission: float
    description: str
    price: float
    quantity: float
    symbol: str
    trade_type: str


class RawTransaction(BaseModel):
    """Raw fields for transactions of type 'trade'."""

    amount: float
    date: datetime
    type: str
    trade: RawTrade


class FlatTransaction(BaseModel):
    """Flat transactions constructed from their raw form."""

    date: datetime
    trade_type: str
    description: str
    symbol: str
    price: float
    quantity: float
    amount: float
    commission: float

    @classmethod
    def flatten_raw_and_validate(cls, transaction: RawTransaction) -> Self:
        """Flatten raw transactions and validate with pydantic model.

        Parameters
        ----------
        transaction : RawTransaction
            Raw transaction object.

        Returns
        -------
        Self
            An instance of the class with validated data.
        """
        # Exclude the `trade` key, this is handled in the following `data.update()` call.
        data = transaction.model_dump(exclude={"trade"})
        data.update(transaction.trade.model_dump())
        return cls.model_validate(data)

In [5]:
def create_equity_dataframe(transactions: list[FlatTransaction]) -> DataFrame:
    """Create and clean a DataFrame containing equity trades.

    Parameters
    ----------
    transactions : list[FlatTransaction]
        Equity trade transactions.

    Returns
    -------
    DataFrame
        Resulting equity DataFrame.
    """
    equity_dump = [transaction.model_dump() for transaction in transactions]
    equity = pd.DataFrame(equity_dump)

    # Convert to more optimal data types.
    equity["trade_type"] = equity["trade_type"].astype(pd.StringDtype())
    equity["description"] = equity["description"].astype(pd.StringDtype())
    equity["symbol"] = equity["symbol"].astype(pd.StringDtype())

    # Add a calculated amount column to compare to the raw `amount` column.
    equity["calc_amount"] = equity["price"] * equity["quantity"] + equity["commission"]
    return equity

In [6]:
def create_options_dataframe(transactions: list[FlatTransaction]) -> DataFrame:
    """Create and clean a DataFrame containing options trades.

    Parameters
    ----------
    transactions : list[FlatTransaction]
        Options trade transactions.

    Returns
    -------
    DataFrame
        Resulting options DataFrame.
    """
    options_dump = [transaction.model_dump() for transaction in transactions]
    options = pd.DataFrame(options_dump)

    # Convert to more optimal data types.
    options["trade_type"] = options["trade_type"].astype(pd.StringDtype())
    options["description"] = options["description"].astype(pd.StringDtype())
    options["symbol"] = options["symbol"].astype(pd.StringDtype())

    # Rename the `symbol` column. A symbol column will be derived from the `description` column.
    options = options.rename(columns={"symbol": "transaction_symbol"})

    # Process the `description` column by splitting the string and creating new columns.
    description_split = options["description"].str.split(expand=True)
    description_split = description_split.drop(2, axis=1)
    description_split = description_split.rename(
        columns={0: "multi_leg_type", 1: "symbol", 3: "multi_leg_amount"}
    )
    description_split["multi_leg_type"] = description_split[
        "multi_leg_type"
    ].str.lower()
    description_split["multi_leg_amount"] = description_split[
        "multi_leg_amount"
    ].astype("float64")

    options = pd.concat([options, description_split], axis=1)

    options["multi_leg_position"] = "long"
    options.loc[options["quantity"] < 0, "multi_leg_position"] = "short"
    options["multi_leg_position"] = options["multi_leg_position"].astype(
        pd.StringDtype()
    )

    # Reorder the columns with numeric columns at the end.
    options_cols = [
        "date",
        "trade_type",
        "symbol",
        "transaction_symbol",
        "price",
        "quantity",
        "amount",
        "commission",
        "multi_leg_position",
        "multi_leg_type",
        "multi_leg_amount",
    ]
    options = options[options_cols]

    options = options.sort_values(by=["date", "symbol"], ascending=[False, True])
    options = options.reset_index(drop=True)

    return options

## Load Data
### Paths

In [7]:
backend_dir = Path.cwd().parent
data_dir = backend_dir / "data"

### Load

In [8]:
history_raw = read_api_data(endpoint_name="history", data_dir=data_dir)
history_raw = history_raw["history"]["event"]

## Transform
### Split Transactions
Source Variable:
- `history_raw`: raw history of all transactions

Resulting Variables:
- `equity_raw`: equity transactions
- `options_raw`: options transactions
- `overrides_raw`: overrides transactions (transactions external to tradier.com)
- `exercised_raw`: exercised transactions
- `other_raw`: any remaining transactions that don't fit the categories above

In [9]:
tradier_trades_raw, overrides_raw, other_raw = split_history(history_raw)

In [10]:
print("Number of each transaction type")
print(f"Equity:    {len(tradier_trades_raw['equity'])}")
print(f"Options:   {len(tradier_trades_raw['options'])}")
print(f"Overrides: {len(overrides_raw)}")
print(f"Other:     {len(other_raw)}")

Number of each transaction type
Equity:    37
Options:   78
Overrides: 12
Other:     1


### Validate with Pydantic Models

In [11]:
tradier_trades = {}
for trade_type, transactions_raw in tradier_trades_raw.items():
    raw_transactions = []
    flat_transactions = []
    for transaction in transactions_raw:
        raw_transaction = RawTransaction.model_validate(transaction)
        raw_transactions.append(raw_transaction)
        flat_transaction = FlatTransaction.flatten_raw_and_validate(raw_transaction)
        flat_transactions.append(flat_transaction)
    tradier_trades[f"{trade_type}_raw"] = raw_transactions
    tradier_trades[trade_type] = flat_transactions

### Create DataFrames
#### Equities

In [12]:
equity = create_equity_dataframe(tradier_trades["equity"])
equity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype              
---  ------       --------------  -----              
 0   date         37 non-null     datetime64[ns, UTC]
 1   trade_type   37 non-null     string             
 2   description  37 non-null     string             
 3   symbol       37 non-null     string             
 4   price        37 non-null     float64            
 5   quantity     37 non-null     float64            
 6   amount       37 non-null     float64            
 7   commission   37 non-null     float64            
 8   calc_amount  37 non-null     float64            
dtypes: datetime64[ns, UTC](1), float64(5), string(3)
memory usage: 2.7 KB


In [13]:
equity.head()

,date,trade_type,description,symbol,price,quantity,amount,commission,calc_amount
0,2023-10-05 00:00:00+00:00,equity,NIKOLA CORPORATION,NKLA,1.360200,-600.0,816.02,0.0,-816.1200
1,2023-10-04 00:00:00+00:00,equity,NIKOLA CORPORATION,NKLA,1.428717,600.0,-857.23,0.0,857.2302
2,2023-10-04 00:00:00+00:00,equity,NIKOLA CORPORATION,NKLA,1.400200,-600.0,840.02,0.0,-840.1200
3,2023-10-03 00:00:00+00:00,equity,NIKOLA CORPORATION,NKLA,1.419700,600.0,-851.82,0.0,851.8200
4,2023-09-22 00:00:00+00:00,equity,MGM RESORTS INTERNATIONAL,MGM,38.000000,200.0,-7609.00,9.0,7609.0000


#### Options

In [14]:
options = create_options_dataframe(tradier_trades["options"])
options.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78 entries, 0 to 77
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   date                78 non-null     datetime64[ns, UTC]
 1   trade_type          78 non-null     string             
 2   symbol              78 non-null     string             
 3   transaction_symbol  78 non-null     string             
 4   price               78 non-null     float64            
 5   quantity            78 non-null     float64            
 6   amount              78 non-null     float64            
 7   commission          78 non-null     float64            
 8   multi_leg_position  78 non-null     string             
 9   multi_leg_type      78 non-null     string             
 10  multi_leg_amount    78 non-null     float64            
dtypes: datetime64[ns, UTC](1), float64(5), string(5)
memory usage: 6.8 KB


In [15]:
options.head()

,date,trade_type,symbol,transaction_symbol,price,quantity,amount,commission,multi_leg_position,multi_leg_type,multi_leg_amount
0,2023-10-06 00:00:00+00:00,option,SPY,SPY231006C00427000,3.35,-3.0,1004.78,0.0,short,call,427.0
1,2023-10-06 00:00:00+00:00,option,SPY,SPY231006P00422000,0.01,3.0,-3.20,0.0,long,put,422.0
2,2023-10-06 00:00:00+00:00,option,SPY,SPY231006P00420000,0.01,-3.0,2.78,0.0,short,put,420.0
3,2023-10-06 00:00:00+00:00,option,SPY,SPY231006C00425000,5.35,3.0,-1605.20,0.0,long,call,425.0
4,2023-10-06 00:00:00+00:00,option,USO,USO231006P00081000,6.88,1.0,-688.07,0.0,long,put,81.0
